In [ ]:
import os

STATIC_WEB_PAGE = {"EXECUTE_NB", "READTHEDOCS"}.intersection(os.environ)

```{autolink-concat}
```

::::{margin}
:::{card} Single-channel T-matrix 3D Riemann sheets
TR-970
^^^
Follow-up to [TR-004](004.ipynb), where we investigate and reproduce the Riemann sheets shown in [Fig.&nbsp;50.1](https://pdg.lbl.gov/2023/reviews/rpp2023-rev-resonances.pdf#page=2) and [50.2](https://pdg.lbl.gov/2023/reviews/rpp2023-rev-resonances.pdf#page=4) of the PDG.
+++
🚧&nbsp;[compwa.github.io#204](https://github.com/ComPWA/compwa.github.io/pull/204)
:::
::::

# Riemann sheets for single-channel T matrix

In [ ]:
%pip install -q ampform==0.14.8 plotly==5.18.0 sympy==1.12

In [ ]:
from __future__ import annotations

import warnings
from typing import Any

import matplotlib.pyplot as plt
import numpy as np
import plotly.graph_objects as go
import sympy as sp
from ampform.io import aslatex
from ampform.sympy import unevaluated
from IPython.display import Math

warnings.filterwarnings("ignore")

## Phase space factor definitions

In [ ]:
from ampform.kinematics.phasespace import Kallen


@unevaluated(real=False)
class PhaseSpaceFactor(sp.Expr):
    s: Any
    m1: Any
    m2: Any
    _latex_repr_ = R"\rho_{{{m1}, {m2}}}\left({s}\right)"

    def evaluate(self) -> sp.Expr:
        s, m1, m2 = self.args
        return 2 * BreakupMomentum(s, m1, m2) / sp.sqrt(s)


@unevaluated(real=False)
class PhaseSpaceCM(sp.Expr):
    s: Any
    m1: Any
    m2: Any
    _latex_repr_ = R"\rho^\mathrm{{CM}}_{{{m1},{m2}}}\left({s}\right)"

    def evaluate(self) -> sp.Expr:
        s, m1, m2 = self.args
        return -16 * sp.pi * sp.I * ChewMandelstam(s, m1, m2)


@unevaluated(real=False)
class ChewMandelstam(sp.Expr):
    s: Any
    m1: Any
    m2: Any
    _latex_repr_ = R"\Sigma\left({s}\right)"

    def evaluate(self) -> sp.Expr:
        s, m1, m2 = self.args
        q = BreakupMomentum(s, m1, m2)
        return (
            1
            / (16 * sp.pi**2)
            * (
                (2 * q / sp.sqrt(s))
                * sp.log((m1**2 + m2**2 - s + 2 * sp.sqrt(s) * q) / (2 * m1 * m2))
                - (m1**2 - m2**2) * (1 / s - 1 / (m1 + m2) ** 2) * sp.log(m1 / m2)
            )
        )


@unevaluated(real=False)
class BreakupMomentum(sp.Expr):
    s: Any
    m1: Any
    m2: Any
    _latex_repr_ = R"q\left({s}\right)"

    def evaluate(self) -> sp.Expr:
        s, m1, m2 = self.args
        return sp.sqrt(Kallen(s, m1**2, m2**2)) / (2 * sp.sqrt(s))


s, m1, m2 = sp.symbols("s m1 m2")
rho_expr = PhaseSpaceFactor(s, m1, m2)
rho_cm_expr = PhaseSpaceCM(s, m1, m2)
cm_expr = ChewMandelstam(s, m1, m2)
q_expr = BreakupMomentum(s, m1, m2)
kallen = Kallen(*sp.symbols("x:z"))
Math(
    aslatex({
        e: e.doit(deep=False)
        for e in [rho_expr, rho_cm_expr, cm_expr, q_expr, kallen]
    })
)

In [ ]:
args = (s, m1, m2)
rho_cm_func = sp.lambdify(args, rho_cm_expr.doit())
rho_func = sp.lambdify(args, rho_expr.doit())

epsilon = 1e-8
s_values = np.linspace(0, 2, num=400)
sp_values = s_values + epsilon * 1j
m1_val = 0.6
m2_val = 0.4
args = (m1_val, m2_val)

cm_values = rho_cm_func(sp_values, *args)
rho_values = rho_func(sp_values, *args)

fig, axes = plt.subplots(figsize=(12, 4), ncols=2, sharex=True, sharey=True)
ax1, ax2 = axes
for ax in axes:
    ax.set_xlabel("Re(s)")
ax1.plot(s_values, cm_values.real, label="Real part")
ax1.plot(s_values, cm_values.imag, label="Imaginary part")
ax1.set_ylabel("Function value")
ax1.set_title(f"${sp.latex(rho_cm_expr)}$")
ax1.legend()

ax2.plot(s_values, rho_values.real, label="Real part")
ax2.plot(s_values, rho_values.imag, label="Imaginary part")
ax2.set_title(f"${sp.latex(rho_expr)}$")
ax2.legend()

ax2.set_ylim(-0.3, +1.2)
plt.tight_layout()

plt.show()

## Define parameters values for plotting 

## T matrix definition with K matrix

In [ ]:
n_channels = 1
I = sp.Identity(n_channels)
K = sp.MatrixSymbol("K", n_channels, n_channels)
rho = sp.MatrixSymbol("rho", n_channels, n_channels)

In [ ]:
T1 = (I - sp.I * K * rho).inv() * K
T1

In [ ]:
T2 = (T1.inv() + 2 * sp.I * rho).inv()

In [ ]:
T1_explicit = T1.as_explicit()
T2_explicit = T2.as_explicit()
T1_explicit[0, 0]

In [ ]:
gamma, m0, w0 = sp.symbols("gamma m0 Gamma0")
k_expr = (gamma**2 * m0 * w0) / (s - m0**2)
definitions = {
    K[0, 0]: k_expr,
    rho[0, 0]: PhaseSpaceCM(s, m1, m2),
}
Math(aslatex(definitions))

In [ ]:
T1_expr = T1_explicit[0, 0].xreplace(definitions)
T2_expr = T2_explicit[0, 0].xreplace(definitions)

In [ ]:
T1_expr.simplify(doit=False)

In [ ]:
T2_expr.simplify(doit=False)

## Riemann sheet visualization

To get to the physical/unphysical Riemann sheet one uses XXXX which is derived from Schwarz reflection principle

In [ ]:
symbols = sp.Tuple(s, m1, m2, m0, w0, gamma)
T1_func = sp.lambdify(symbols, T1_expr.doit())
T2_func = sp.lambdify(symbols, T2_expr.doit())

In [ ]:
epsilon = 1e-5
x = np.linspace(0, 6, num=200)
y = np.linspace(epsilon, 1, num=100)
X, Y = np.meshgrid(x, y)
Zn = X - Y * 1j
Zp = X + Y * 1j

values = {
    m1: 0.9,
    m2: 0.8,
    m0: 3.1,
    w0: 1.0,
    gamma: 1,
}
args = eval(str(symbols[1:].xreplace(values)))

T1n = T1_func(Zn**2, *args)
T1p = T1_func(Zp**2, *args)

T2n = T2_func(Zn**2, *args)
T2p = T2_func(Zp**2, *args)

In [ ]:
fig, axes = plt.subplots(figsize=(15, 6), ncols=2, sharey=True)
ax1, ax2 = axes
for ax in axes:
    ax.set_xlabel(R"$\mathrm{Re}(s)$")

ax1.plot(x, T1n[0].imag, label=R"$T_\mathrm{I}(s-0i)$")
ax1.plot(x, T1p[0].imag, label=R"$T_\mathrm{I}(s+0i)$")
ax1.set_title(f"${sp.latex(rho_cm_expr)}$")
ax1.set_title(R"$T_\mathrm{I}$")

ax2.plot(x, T2n[0].imag, label=R"$T_\mathrm{II}(s-0i)$")
ax2.plot(x, T2p[0].imag, label=R"$T_\mathrm{II}(s+0i)$")
ax2.set_title(R"$T_\mathrm{II}$")

for ax in axes:
    ax.legend()

fig.tight_layout()
plt.show()

In [ ]:
def sty(sheet_name: str) -> dict:
    sheet_color = sheet_colors[sheet_name]
    n_lines = 16
    return dict(
        cmin=-vmax,
        cmax=+vmax,
        colorscale=[[0, "rgb(0, 0, 0)"], [1, sheet_color]],
        contours=dict(
            x=dict(
                show=True,
                start=x.min(),
                end=x.max(),
                size=(x.max() - x.min()) / n_lines,
                color="black",
            ),
            y=dict(
                show=True,
                start=-y.max(),
                end=+y.max(),
                size=(y.max() - y.min()) / (n_lines // 2),
                color="black",
            ),
        ),
        name=sheet_name,
        opacity=0.6,
        showscale=False,
    )


vmax = 1.6
project = np.imag
sheet_colors = {
    "Unphysical (T1)": "blue",
    "Physical (T2)": "red",
}

In [ ]:
Sn = go.Surface(x=X, y=-Y, z=T2n.imag, **sty("Physical (T2)"))
Sp = go.Surface(x=X, y=+Y, z=T1p.imag, **sty("Unphysical (T1)"))
zeros = np.zeros(x.shape)
lineshape = go.Scatter3d(
    x=x,
    y=zeros,
    z=project(T1p[0]),
    line=dict(color="yellow", width=10),
    mode="lines",
    name="Lineshape",
)

fig = go.Figure(data=[Sn, Sp, lineshape])
fig.update_layout(
    height=550,
    margin=dict(l=0, r=0, t=30, b=0),
    showlegend=False,
    title_text="Im(T) with Chew-Mandelstam phase space factor",
)
fig.update_scenes(
    camera_center=dict(z=-0.2),
    xaxis_title_text="Re s",
    yaxis_title_text="Im s",
    zaxis_range=[-vmax, +vmax],
)

fig.show()